In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [30]:
# 데이터 로드
train = pd.read_csv(r"C:\Users\DELL\OneDrive\바탕 화면\학교\대학교 3학년\LG aimers\open\train.csv", encoding="utf-8")
test = pd.read_csv(r"C:\Users\DELL\OneDrive\바탕 화면\학교\대학교 3학년\LG aimers\open\test.csv", encoding="utf-8")

# 선택할 feature
selected_features = ["시술 당시 나이", "시술 유형", "총 시술 횟수", "IVF 임신 횟수", "IVF 출산 횟수", "DI 임신 횟수", "DI 출산 횟수",
                     "여성 주 불임 원인", "여성 부 불임 원인", "남성 주 불임 원인", "남성 부 불임 원인", "부부 주 불임 원인", "부부 부 불임 원인",
                     "불임 원인 - 난관 질환", "불임 원인 - 배란 장애", "불임 원인 - 남성 요인", "불명확 불임 원인", "불임 원인 - 자궁내막증", "불임 원인 - 자궁경부 문제",
                     "불임 원인 - 정자 농도", "불임 원인 - 정자 형태", "불임 원인 - 정자 운동성", "신선 배아 사용 여부", "동결 배아 사용 여부", "해동된 배아 수",
                     "수집된 신선 난자 수", "파트너 정자와 혼합된 난자 수", "이식된 배아 수"]


In [31]:
target_column = "임신 성공 여부"
train = train[selected_features + [target_column]]
test = test[selected_features]

In [ ]:
def preprocess_data(df, is_train=True):
    # 레이블 인코딩
    label_encoder = LabelEncoder()
    df['시술 유형'] = label_encoder.fit_transform(df['시술 유형'])
    
    # 결측값 처리
    di_columns = ['해동된 배아 수', '수집된 신선 난자 수', '파트너 정자와 혼합된 난자 수', '이식된 배아 수', "신선 배아 사용 여부", "동결 배아 사용 여부"]
    df.loc[df['시술 유형'] == 0, di_columns] = df.loc[df['시술 유형'] == 0, di_columns].fillna(0)
    
    # "시술 당시 나이" 변환
    age_mapping = {"만18-34세": 0, "만35-37세": 1, "만38-39세": 2, "만40-42세": 3, "만43-44세": 4, "만45-50세": 5}
    df['시술 당시 나이'] = df['시술 당시 나이'].map(age_mapping).fillna(np.nan)
    df['시술 당시 나이'].fillna(df['시술 당시 나이'].mean(), inplace=True)
    
    # 시술 횟수 변환 함수
    def convert_string_to_num(count):
        mapping = {"0회": 0, "1회": 1, "2회": 2, "3회": 3, "4회": 4, "5회": 5, "6회 이상": 6}
        return mapping.get(count, None)
    
    count_cols = ["총 시술 횟수", "IVF 임신 횟수", "IVF 출산 횟수", "DI 임신 횟수", "DI 출산 횟수"]
    for col in count_cols:
        df[col] = df[col].apply(convert_string_to_num)
    
    # 정규화
    scaler = MinMaxScaler()
    normalize_cols = ["시술 당시 나이", "IVF 임신 횟수", "IVF 출산 횟수", "DI 임신 횟수", "DI 출산 횟수", "이식된 배아 수", "해동된 배아 수", "수집된 신선 난자 수", "파트너 정자와 혼합된 난자 수"]
    df[normalize_cols] = scaler.fit_transform(df[normalize_cols])
    
    return df

In [34]:
# 데이터 전처리
train = preprocess_data(train)
test = preprocess_data(test, is_train=False)

In [35]:
# 훈련 데이터 분할
x = train.drop(columns=[target_column])
y = train[target_column]

In [36]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42, stratify=y)

In [37]:
# 모델 학습
model = LogisticRegression()
model.fit(x_train, y_train)

c:\Users\DELL\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [38]:
# 검증 데이터 평가
y_pred = model.predict(x_val)
y_prob = model.predict_proba(x_val)[:, 1]
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Classification Report:\n", classification_report(y_val, y_pred))

Accuracy: 0.7400592931630822
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.99      0.85     57037
           1       0.43      0.02      0.04     19869

    accuracy                           0.74     76906
   macro avg       0.59      0.50      0.44     76906
weighted avg       0.66      0.74      0.64     76906



In [41]:
# 테스트 데이터 예측
test_ids = [f"TEST_{str(i).zfill(5)}" for i in range(len(test))]  # ID 컬럼이 없을 경우 인덱스 사용
y_test_prob = model.predict_proba(test)[:, 1]

In [42]:
# 결과 저장
result = pd.DataFrame({
    'ID': test_ids,
    'probability': y_test_prob
})
result.to_csv('prediction_results2.csv', index=False, encoding='utf-8-sig')

0.666